<a href="https://colab.research.google.com/github/JosnaKurian/Extraction_of_tabular_data_from_PDF/blob/main/pdf_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EXTRACTION OF TABULAR DATA FROM PDF

#### Problem statement:

Read the PDFs using python and extract the table contents/Line items as a DATAFRAME from the PDFs.

Find a method/ strategy/ algorithm/ pre-trained models to identify the table contents, for all PDFs given and implement in Python.

Extract all the columns as close as possible to the PDF given.



In [2]:
# Installing required packages

!pip install pdf2image   # Used to convert pdf to image
!apt-get install poppler-utils
!pip install layoutparser   # Used to convert pdf to image
!pip install paddleocr   # Used for detection and recoginition of text
!pip install paddlepaddle

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (443 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [3]:
# converting the pdf into image

from pdf2image import convert_from_path
images = convert_from_path('/content/Pdf_1.pdf')

In [4]:
!mkdir pages

In [5]:
# Save the images in the pages folder

for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg','JPEG')

Installing Detectron2.

Detectron2 is a framework for object detection.

In [6]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.4) to /tmp/pip-install-8jjgkt65/detectron2_ce6a51cae66a4a62a2a71c2fe831f43b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-8jjgkt65/detectron2_ce6a51cae66a4a62a2a71c2fe831f43b
  Running command git checkout -q 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.4-cp310-cp310-linux_x86_64.whl size=5641910 sha256=57c4590ee7f55

Detectron2LayoutModel: This is a layout analysis model provided by the layoutparser library. It utilizes the Detectron2 framework.

'lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config': This is the configuration used for the underlying Detectron2 model.

label_map is provided to map the class indices predicted by the model to human-readable class labels.

In [7]:
import layoutparser as lp
model = lp.models.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',
                                        label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

config.yaml?dl=1: 8.19kB [00:00, 10.0kB/s]
model_final.pth?dl=1: 856MB [00:08, 105MB/s]                           


In [8]:
import cv2
image = cv2.imread('/content/pages/page0.jpg')  # reading the image using OpenCV
image = image[...,::-1]   # reverses the order of color channels in the image

layout = model.detect(image)   # detects regions of interest

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=49.321311950683594, y_1=1889.24072265625, x_2=1614.038330078125, y_2=1990.5751953125), text=None, id=None, type=Text, parent=None, next=None, score=0.9837935566902161), TextBlock(block=Rectangle(x_1=1360.932861328125, y_1=1829.448486328125, x_2=1514.9866943359375, y_2=1864.7947998046875), text=None, id=None, type=Title, parent=None, next=None, score=0.9490137696266174), TextBlock(block=Rectangle(x_1=432.7635803222656, y_1=2013.5079345703125, x_2=1273.029052734375, y_2=2042.128173828125), text=None, id=None, type=Title, parent=None, next=None, score=0.89915931224823), TextBlock(block=Rectangle(x_1=121.7755126953125, y_1=746.740234375, x_2=1614.835693359375, y_2=1628.4481201171875), text=None, id=None, type=Table, parent=None, next=None, score=0.6938751935958862), TextBlock(block=Rectangle(x_1=103.02120208740234, y_1=209.83416748046875, x_2=660.0382080078125, y_2=274.39520263671875), text=None, id=None, type=Text, parent=None, next=None, scor

In [10]:
# Finding the co-ordinates of the table layout

for l in layout:
  if l.type == 'Table':
    x_1 = int(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)
    break

In [11]:
# Saving the extracted table image

img = cv2.imread('/content/pages/page0.jpg')
cv2.imwrite('extracted_img.jpg',img[y_1:y_2, x_1:x_2])

True

Cloning the PaddleOCR git repo

In [12]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 47625, done.
remote: Counting objects: 100% (866/866), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 47625 (delta 558), reused 683 (delta 472), pack-reused 46759
Receiving objects: 100% (47625/47625), 345.27 MiB | 27.64 MiB/s, done.
Resolving deltas: 100% (33410/33410), done.
Updating files: 100% (1892/1892), done.


In [13]:
%cd PaddleOCR/ppstructure

/content/PaddleOCR/ppstructure


In [14]:
!mkdir inference
%cd inference

/content/PaddleOCR/ppstructure/inference


In [15]:
# Download the PP-OCRv3 detection model and unzip it
!wget https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar && tar xf en_PP-OCRv3_det_infer.tar

# Download the PP-OCRv3 text recognition model and unzip it
!wget https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar && tar xf en_PP-OCRv3_rec_infer.tar

# Download the PP-StructureV2 form recognition model and unzip it
!wget https://paddleocr.bj.bcebos.com/ppstructure/models/slanet/en_ppstructure_mobile_v2.0_SLANet_infer.tar && tar xf en_ppstructure_mobile_v2.0_SLANet_infer.tar

--2023-12-03 17:08:15--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4003840 (3.8M) [application/x-tar]
Saving to: ‘en_PP-OCRv3_det_infer.tar’

en_PP-OCRv3_det_inf 100%[===================>]   3.82M   730KB/s    in 14s     

2023-12-03 17:08:30 (286 KB/s) - ‘en_PP-OCRv3_det_infer.tar’ saved [4003840/4003840]

--2023-12-03 17:08:30--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9963520 (9.5M) [application/

In [18]:
# Run the predict_table.py with detection, recognition and table structure model

!python3 /content/PaddleOCR/ppstructure/table/predict_table.py --det_model_dir=/content/PaddleOCR/ppstructure/inference/en_PP-OCRv3_det_infer --rec_model_dir=/content/PaddleOCR/ppstructure/inference/en_PP-OCRv3_rec_infer --table_model_dir=/content/PaddleOCR/ppstructure/inference/en_ppstructure_mobile_v2.0_SLANet_infer --image_dir=/content/extracted_img.jpg --rec_char_dict_path=/content/PaddleOCR/ppocr/utils/en_dict.txt --table_char_dict_path=/content/PaddleOCR/ppocr/utils/dict/table_structure_dict.txt --det_limit_side_len=736 --det_limit_type=min --output ./output/table

E1203 17:09:49.033385 13674 analysis_config.cc:121] Please use PaddlePaddle with GPU version.
E1203 17:09:49.509593 13674 analysis_config.cc:121] Please use PaddlePaddle with GPU version.
E1203 17:09:49.764001 13674 analysis_config.cc:121] Please use PaddlePaddle with GPU version.
E1203 17:09:50.446381 13674 analysis_config.cc:121] Please use PaddlePaddle with GPU version.
[2023/12/03 17:09:51] ppocr INFO: [0/1] /content/extracted_img.jpg
[2023/12/03 17:09:55] ppocr DEBUG: dt_boxes num : 81, elapse : 2.691648483276367
[2023/12/03 17:10:02] ppocr DEBUG: rec_res num  : 81, elapse : 7.025516986846924
[2023/12/03 17:10:02] ppocr INFO: <html><body><table><thead><tr><td colspan="2">Tracking: Customer PO 57265</td><td>UPS/1Z 724 251 03 5206 0749 Payment Terms</td><td>Invoice Due Ship To PO Date 13-FEB-22</td><td colspan="2">invoice Currency USD</td><td>Incoterm FOB ORIGIN</td><td colspan="2">Sales Order 4582446</td></tr><tr><td>e</td><td>Item</td><td>Net 45</td><td>Description</td><td></td><t

### Convert the excel file into DataFrame

In [19]:
import pandas as pd
df = pd.read_excel('/content/PaddleOCR/ppstructure/inference/output/table/extracted_img.xlsx')
df

,Tracking: Customer PO 57265,Unnamed: 1,UPS/1Z 724 251 03 5206 0749 Payment Terms,Invoice Due Ship To PO Date 13-FEB-22,invoice Currency USD,Unnamed: 5,Incoterm FOB ORIGIN,Sales Order 4582446,Unnamed: 8
0,e,Item,Net 45,Description,NaN,UOM Qty,Unit Net,Price,Extended Price
1,1,NT8501376,Lot# 5961218(13),"Open-Ended Peritoneal Catheter w/Slits, 120cm",EA,NaN,13,39.86,518.18
2,2,NT8501229,NaN,"Portnoy Flanged Ventricular Catheter, 18cm",EA,8,NaN,29.69,237.52
3,3,NT8500112,NaN,Lot# 5961197(8) Natus Multi-Purpose Valve wAnt...,EA,1,NaN,128.97,128.97
4,4,NT97001S01,Lot# 5905603(1) Luer Connector,NaN,EA,5,NaN,3.01,15.05
5,5,NT8501195,Lot# 5949500(5),FOLTZ FLUSHING RESERVOIR,EA,1,NaN,34.14,34.14
6,6,NT83024S01,Lot# 5958671(1),STERILE VENTRICULAR CATHETER GUIDE,EA,11,NaN,14.68,161.48
7,7,NT8501391,Lot# 5961229(11) Lot# 5961226(3),"Pudenz Peritoneal Catheter Barium Imp Stripe, MP",EA,3,NaN,25.95,77.85
